In [ ]:
#####################################################################################
# Audio-driven upper-body motion synthesis on a humanoid robot
# Computer Science Tripos Part III Project
# Jan Ondras (jo356@cam.ac.uk), Trinity College, University of Cambridge
# 2017/18
#####################################################################################
# Render 3D poses (with labeled body joints) reconstructed by each method (OP+M, OP+A, LFTD, VNect)
#####################################################################################

In [40]:
#########################################################################################################
# Render 3D just one frame with joints' labels
# OP+A (same skeleton as for OP+C)
#########################################################################################################

from geoutils import radToDeg, xyz_to_angles
from vpython import *
from scipy.io import loadmat
import numpy as np
from geo import *

VID = 'PID20Task2'
# VID = 'PID23Task3'

#################################################

joints_used = [0,1,2,3,4,5,6,7, 8, 11] # 14 from COCO model, w/o legs, eyes, ears

joints_names = ['Head','Neck','Right shoulder','Right elbow','Right wrist',
                'Left shoulder','Left elbow','Left wrist',
                'Right hip','right_knee','right_ankle',
                'Left hip','left_knee','left_ankle',
                'right_eye','left_eye','right_ear', 'left_ear']
joint_connections = [
    (0, 1),                                                          # central, blue
    (1,8), (1,11),                                                   # extra for Openpose
    (1, 2), (2, 3), (3, 4), (8, 9), (9, 10), (0, 14), (14, 16),      # right, red
    (1, 5), (5, 6), (6, 7), (11, 12), (12, 13), (0, 15), (15, 17)    # left, green
]
joint_connections_colors = [color.blue]*3 + [color.red]*7 + [color.green]*7
joints_colors = [color.white]*len(joints_names)

data = np.load('./../Dataset/OpenPoseConstraintsFeatures/' + VID + '.npz')['joints_3D']

print ("Shape =" , np.shape(data))

# Setup scene
scene = canvas()
scene.width = 960
scene.height = 600
scene.title = "Openpose 3D pose reconstruction using Constraints and assumptions"
scene.scale = 400.
# scene.fov = np.pi /3.
scene.up=vector(0.,-1.,0.)     # To flip scene horiznotally !!!
scene.forward=vector(0.,0.,1.)     # To rotate around y !!!
# scene.center=vector(0.,-150.,10000) # To shift down a but

###################################### Centering to neck joint
data[0, :, :] = data[0, :, :] - data[0, 1, :]

scene.lights = []
scene.ambient=color.gray(0.8)
frame = 0
scene.caption = 'Frame: ' + str(frame)
Ps = []
Es = []
Ls = []
# Draw all joints requested
for i in joints_used:
    x = data[frame, i, 0] #- skeleton_center[0]
    y = data[frame, i, 1] #- skeleton_center[1]
    z = data[frame, i, 2] #- skeleton_center[2]
    P = sphere(pos=vector(x,y,z), radius=10, color=joints_colors[i])#, #size_units="world")
    Ps.append(P)
    
    if i in [2,3,4]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-30,
                yoffset=30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [5,6,7]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=30,
                yoffset=30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [8]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-10,
                yoffset=-30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [11]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=10,
                yoffset=-30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [1]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-35,
                yoffset=40, space=12,box=False,
                height=16, #border=0,
                font='sans')
    else: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-5,
                yoffset=30, space=12,
                height=16, #border=0,
                  box=False,
                font='sans')
    Ls.append(L)

# Draw all links between joints
for i, (a,b) in enumerate(joint_connections):
    if a in joints_used and b in joints_used:
        ax = data[frame, a, 0] #- skeleton_center[0]
        ay = data[frame, a, 1] #- skeleton_center[1]
        az = data[frame, a, 2] #- skeleton_center[2]
        bx = data[frame, b, 0] #- skeleton_center[0]
        by = data[frame, b, 1] #- skeleton_center[1]
        bz = data[frame, b, 2] #- skeleton_center[2]
        cx = (ax + bx) / 2.
        cy = (ay + by) / 2.
        cz = (az + bz) / 2.
        E = ellipsoid(pos=vector(cx,cy,cz), axis=vector(ax-bx,ay-by,az-bz), length=np.linalg.norm([ax-bx,ay-by,az-bz]), 
                  height=25, width=25, color=joint_connections_colors[i])
        Es.append(E)


Shape = (1302, 18, 3)


<IPython.core.display.Javascript object>

In [66]:
#########################################################################################################
# Render 3D just one frame with joints' labels
# LFTD
#####################################

from geoutils import radToDeg, xyz_to_angles
from vpython import *
from scipy.io import loadmat
import numpy as np
from pykalman import KalmanFilter
from geo import *

VID = 'PID20Task2'
VID = 'PID23Task3'

#################################################

joints_used = np.arange(17) # first 17
joints_used = [0,1,  4,  7, 8, 9, 10, 11, 12, 13, 14,15,16] # first 17, w/o legs

joints_names = ['Pelvis', 'Right hip', '', '', 'Left hip', '', '', 'Spine', 'Neck', 'Nose', 'Head', 
                'Left shoulder','Left elbow','Left wrist',
                'Right shoulder','Right elbow','Right wrist']

joint_connections = [
        [0, 1], [1, 2], [2, 3], [0, 4], [4, 5], [5, 6], [0, 7], [7, 8],
        [8, 9], [9, 10], [8, 11], [11, 12], [12, 13], [8, 14], [14, 15],
        [15, 16]
]
# joint_connections_colors = [color.red]*3 + [color.green]*3 + [color.blue]*4 + [color.green]*3 + [color.red]*3
joint_connections_colors = [color.blue]*10 + [color.green]*3 + [color.red]*3
joints_colors = [color.white]*len(joints_names)

smooth_joints_pos = False
if smooth_joints_pos:
    data = np.load('./../Dataset/LiftFromDeep_smoothed/' + VID + '.npz')['joints_3D']
else:
    data = np.load('./../Dataset/LiftFromDeep/' + VID + '.npz')['joints_3D']

# print (data)
#     data = np.reshape(data, (-1,17,3))
print ("Shape =" , np.shape(data))

# Setup scene
scene = canvas()
scene.width = 960
scene.height = 600
scene.title = "Lifting from the Deep: 3D pose reconstruction"
scene.scale = 400.
# scene.fov = np.pi /2.
scene.up=vector(0.,1.,1.9)     # To flip scene horiznotally !!!
scene.forward=vector(0.,0.75,-0.6)     # To rotate around y !!!
scene.center=vector(0.,270.,0.) # To shift down a bit

scene.lights = []
scene.ambient=color.gray(0.8)
frame = 0
scene.caption = 'Frame: ' + str(frame)
Ps = []
Es = []
Ls = []
# Draw all joints requested
for i in joints_used:
    x = data[frame, i, 0]
    y = data[frame, i, 1]
    z = data[frame, i, 2]
    P = sphere(pos=vector(x,y,z), radius=10, color=joints_colors[i])#, #size_units="world")
    Ps.append(P)
    if i in [11,12,13]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=30,
                yoffset=30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [14,15,16]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-30,
                yoffset=30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [0]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-0,
                yoffset=-30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [4]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=10,
                yoffset=-30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [1]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-10,
                yoffset=-30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [7]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-25,
                yoffset=20, space=12,box=False,
                height=16, #border=0,
                font='sans')
    else: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-35,
                yoffset=30, space=12,
                height=16, #border=0,
                  box=False,
                font='sans')
    Ls.append(L)
    


# Draw all links between joints
for i, (a,b) in enumerate(joint_connections):
    if a in joints_used and b in joints_used:
        ax = data[frame, a, 0]
        ay = data[frame, a, 1]
        az = data[frame, a, 2]
        bx = data[frame, b, 0]
        by = data[frame, b, 1]
        bz = data[frame, b, 2]     
        cx = (ax + bx) / 2.
        cy = (ay + by) / 2.
        cz = (az + bz) / 2.
        E = ellipsoid(pos=vector(cx,cy,cz), axis=vector(ax-bx,ay-by,az-bz), length=np.linalg.norm([ax-bx,ay-by,az-bz]), 
                  height=25, width=25, color=joint_connections_colors[i])
        Es.append(E)



Shape = (2516, 17, 3)


<IPython.core.display.Javascript object>

In [14]:
#########################################################################################################
# Render 3D just one frame with joints' labels
# VNect
#########################################################################################################

from geoutils import radToDeg, xyz_to_angles
from vpython import *
from scipy.io import loadmat
import numpy as np
from pykalman import KalmanFilter
from geo import *

VID = 'PID05Task3'
# VID = 'PID23Task3'
# VID = 'PID17Task3'
# VID = 'PID20Task3'

# joints_used = np.arange(17) # first 17
joints_used = [0,1,2,3,4,5,6,7, 8, 11, 14,15,16] # first 17, w/o legs

joints_names = ['Head','Neck','Right shoulder','Right elbow','Right wrist','Left shoulder','Left elbow','Left wrist',
                'Right hip','right_knee','right_ankle','Left hip','left_knee','left_ankle',
                'Pelvis','Spine','Nose',
                'right_hand','left_hand','right_toe','left_toe']

joint_connections = [
    (0, 16), (1, 15), (14, 15), (1, 16),       (8, 14), (11, 14),     # central, blue
    (1, 2), (2, 3), (3, 4), (4, 17), (8, 9), (9, 10), (10, 19),       # right, red
    (1, 5), (5, 6), (6, 7), (7, 18), (11, 12), (12, 13), (13, 20),   # left, green
]
joint_connections_colors = [color.blue]*6 + [color.red]*7 + [color.green]*7
joints_colors = [color.white]*len(joints_names)

data = loadmat('./../Dataset/VNectPose/' + VID + '.mat')['Mt']
data = np.reshape(data, (-1,21,3))
print ("Shape =" , np.shape(data))

# Setup scene
scene = canvas()
scene.width = 960
scene.height = 600
scene.title = "VNect 3D pose reconstruction"
scene.scale = 400.
# scene.fov = np.pi /3.
scene.up=vector(0.,-1.,0.)     # To flip scene horiznotally !!!
scene.forward=vector(0.,0.,1.)     # To rotate around y !!!
scene.center=vector(0.,-250.,0.) # To shift down a but

scene.lights = []
scene.ambient=color.gray(0.8)
frame = 100
scene.caption = 'Frame: ' + str(frame)
Ps = []
Es = []
Ls = []
# Draw all joints requested
for i in joints_used:
    x = data[frame, i, 0]
    y = data[frame, i, 1]
    z = data[frame, i, 2]
    P = sphere(pos=vector(x,y,z), radius=10, color=joints_colors[i])#, #size_units="world")
    Ps.append(P)
    if i in [5,6]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=30,
                yoffset=30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [7]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=40,
                yoffset=-30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [2,3]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-30,
                yoffset=30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [4]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-40,
                yoffset=-30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [14]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-0,
                yoffset=-50, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [11]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=10,
                yoffset=-30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [8]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-10,
                yoffset=-30, space=12,box=False,
                height=16, #border=0,
                font='sans')
    elif i in [15]: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-25,
                yoffset=20, space=12,box=False,
                height=16, #border=0,
                font='sans')
    else: 
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=-35,
                yoffset=30, space=12,
                height=16, #border=0,
                  box=False,
                font='sans')
    Ls.append(L)
    
    joints_names = ['Head','Neck','Right shoulder','Right elbow','Right wrist','Left shoulder','Left elbow','Left wrist',
                'Right hip','right_knee','right_ankle','Left hip','left_knee','left_ankle',
                'Pelvis','Spine','Nose',
                'right_hand','left_hand','right_toe','left_toe']

# Draw all links between joints
for i, (a,b) in enumerate(joint_connections):
    if a in joints_used and b in joints_used:
        ax = data[frame, a, 0]
        ay = data[frame, a, 1]
        az = data[frame, a, 2]
        bx = data[frame, b, 0]
        by = data[frame, b, 1]
        bz = data[frame, b, 2]     
        cx = (ax + bx) / 2.
        cy = (ay + by) / 2.
        cz = (az + bz) / 2.
        E = ellipsoid(pos=vector(cx,cy,cz), axis=vector(ax-bx,ay-by,az-bz), length=np.linalg.norm([ax-bx,ay-by,az-bz]), 
                  height=25, width=25, color=joint_connections_colors[i])
        Es.append(E)



Shape = (3914, 21, 3)


<IPython.core.display.Javascript object>